In [58]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras as keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import utils

## Загрузка набора данных

In [59]:
data = pd.read_csv('11.txt', sep='\t', header=0, quotechar='"')
data = data.dropna()


data.head()

,Cr,Ni,Mo,Mn,Si,Nb,Ti,Zr,Ta,V,...,Grains mm-2,Type of melting,Size of ingot,Product form,Temperature (K),0.2%proof_stress (M Pa),UTS (M Pa),Elongation (%),Area_reduction (%),Comments
66,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,298,210.0,620.0,71.0,81.0,NRIM/CDS/No4B/1986 ABL
67,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,373,207.0,503.0,62.0,82.0,NRIM/CDS/No4B/1986 ABL
68,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,473,177.0,465.0,46.0,77.0,NRIM/CDS/No4B/1986 ABL
69,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,573,162.0,468.0,45.0,71.0,NRIM/CDS/No4B/1986 ABL
70,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,238.856446,1.0,4.5,1,673,154.0,463.0,47.0,73.0,NRIM/CDS/No4B/1986 ABL


In [60]:
# y - то что будем предсказывать
# X - данный, по которым предсказывается y

y = data['UTS (M Pa)']

X = data.drop('UTS (M Pa)', axis=1)

# убираем лишнее

X = X.drop('Comments',axis=1)
X = X.drop('0.2%proof_stress (M Pa)',axis=1)
X = X.drop('Grains mm-2',axis=1)
X = X.drop('Type of melting',axis=1)



X.head()





,Cr,Ni,Mo,Mn,Si,Nb,Ti,Zr,Ta,V,...,Pb,Solution_treatment_temperature,Solution_treatment_time(s),Water_Quenched_after_s.t.,Air_Quenched_after_s.t.,Size of ingot,Product form,Temperature (K),Elongation (%),Area_reduction (%)
66,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0,1343.0,600.0,1.0,0.0,4.5,1,298,71.0,81.0
67,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0,1343.0,600.0,1.0,0.0,4.5,1,373,62.0,82.0
68,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0,1343.0,600.0,1.0,0.0,4.5,1,473,46.0,77.0
69,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0,1343.0,600.0,1.0,0.0,4.5,1,573,45.0,71.0
70,18.16,9.8,0.05,1.47,0.58,0.04,0.031,0,0,0.0,...,0,1343.0,600.0,1.0,0.0,4.5,1,673,47.0,73.0


## Разделение датасета на тренировочную и тестовую части

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    # доля объёма тестового множества
                                                    test_size=0.3)



print(X_train)


        Cr     Ni     Mo    Mn    Si     Nb     Ti  Zr  Ta      V  ...  Pb  \
628  18.55   9.00  0.100  0.96  0.63  0.010  0.020   0   0  0.057  ...   0   
560  17.61  12.50  0.060  1.73  0.82  0.790  0.019   0   0  0.032  ...   0   
641  18.53   9.10  0.120  0.88  0.54  0.005  0.020   0   0  0.052  ...   0   
557  17.61  12.50  0.060  1.73  0.82  0.790  0.019   0   0  0.032  ...   0   
311  17.60  11.19  2.320  1.74  0.74  0.030  0.014   0   0  0.000  ...   0   
..     ...    ...    ...   ...   ...    ...    ...  ..  ..    ...  ...  ..   
324  17.43  12.48  2.490  1.49  0.46  0.040  0.009   0   0  0.000  ...   0   
409  21.00  32.34  0.002  1.12  0.43  0.000  0.490   0   0  0.000  ...   0   
461  20.90  34.45  0.110  0.95  0.44  0.000  0.430   0   0  0.000  ...   0   
85   18.24  10.27  0.320  1.56  0.61  0.030  0.036   0   0  0.000  ...   0   
373  17.11  12.30  2.310  1.44  0.67  0.040  0.013   0   0  0.000  ...   0   

     Solution_treatment_temperature  Solution_treatment_time(s)

## Преобразовывание данных в матричный вид

In [62]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

print(X_train)


[[1.855e+01 9.000e+00 1.000e-01 ... 8.730e+02 4.600e+01 7.300e+01]
 [1.761e+01 1.250e+01 6.000e-02 ... 7.730e+02 3.700e+01 6.500e+01]
 [1.853e+01 9.100e+00 1.200e-01 ... 9.230e+02 4.600e+01 7.300e+01]
 ...
 [2.090e+01 3.445e+01 1.100e-01 ... 5.730e+02 4.700e+01 6.700e+01]
 [1.824e+01 1.027e+01 3.200e-01 ... 9.230e+02 4.600e+01 6.500e+01]
 [1.711e+01 1.230e+01 2.310e+00 ... 7.730e+02 4.700e+01 6.900e+01]]


## Нормализация данных

In [63]:

from sklearn.preprocessing import MinMaxScaler 
scale_features_mm = MinMaxScaler() 
X_train = scale_features_mm.fit_transform(X_train) 
X_test = scale_features_mm.transform(X_test) 

print(X_train)


[[0.46252677 0.         0.03831118 ... 0.58974359 0.2991453  0.75641026]
 [0.26124197 0.13752456 0.02267396 ... 0.48717949 0.22222222 0.65384615]
 [0.45824411 0.00392927 0.04612979 ... 0.64102564 0.2991453  0.75641026]
 ...
 [0.96573876 1.         0.04222048 ... 0.28205128 0.30769231 0.67948718]
 [0.39614561 0.04990177 0.12431587 ... 0.64102564 0.2991453  0.65384615]
 [0.15417559 0.12966601 0.9022674  ... 0.48717949 0.30769231 0.70512821]]


In [64]:
from keras import regularizers
from keras.layers import Dropout

model = Sequential()
model.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l1_l2(l1 = 0.001, l2 = 0.001),  input_shape=(X_train.shape[1],)))
model.add(Dense(2048, activation='relu', ))
model.add(Dense(1024, activation='relu', ))
model.add(Dense(2048, activation='relu', ))


model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

## Обучение нейронной сети

In [65]:
model.fit(X_train, y_train, epochs=1500, batch_size=64, verbose=2)

Epoch 1/1500
 - 0s - loss: 179623.5438 - mae: 403.5489
Epoch 2/1500
 - 0s - loss: 119864.6858 - mae: 318.2290
Epoch 3/1500
 - 0s - loss: 55513.7905 - mae: 184.5340
Epoch 4/1500
 - 0s - loss: 31693.4649 - mae: 153.4561
Epoch 5/1500
 - 0s - loss: 40062.6316 - mae: 181.1315
Epoch 6/1500
 - 0s - loss: 22070.6950 - mae: 113.6744
Epoch 7/1500
 - 0s - loss: 23124.3930 - mae: 104.6258
Epoch 8/1500
 - 0s - loss: 18531.3538 - mae: 118.7103
Epoch 9/1500
 - 0s - loss: 15268.4196 - mae: 104.6109
Epoch 10/1500
 - 0s - loss: 13820.6427 - mae: 80.6395
Epoch 11/1500
 - 0s - loss: 10726.5958 - mae: 79.9964
Epoch 12/1500
 - 0s - loss: 9566.6434 - mae: 78.0565
Epoch 13/1500
 - 0s - loss: 7831.0716 - mae: 62.9272
Epoch 14/1500
 - 0s - loss: 6119.1108 - mae: 59.2107
Epoch 15/1500
 - 0s - loss: 5120.5927 - mae: 55.3787
Epoch 16/1500
 - 0s - loss: 4223.9473 - mae: 51.0470
Epoch 17/1500
 - 0s - loss: 3717.9568 - mae: 49.4331
Epoch 18/1500
 - 0s - loss: 3543.3212 - mae: 48.3362
Epoch 19/1500
 - 0s - loss: 3445.

Epoch 156/1500
 - 0s - loss: 345.7495 - mae: 15.1237
Epoch 157/1500
 - 0s - loss: 400.0784 - mae: 15.9467
Epoch 158/1500
 - 0s - loss: 425.9327 - mae: 16.5338
Epoch 159/1500
 - 0s - loss: 427.9515 - mae: 16.8533
Epoch 160/1500
 - 0s - loss: 349.6533 - mae: 14.2772
Epoch 161/1500
 - 0s - loss: 386.1891 - mae: 15.7459
Epoch 162/1500
 - 0s - loss: 367.7933 - mae: 15.6361
Epoch 163/1500
 - 0s - loss: 400.8130 - mae: 15.9772
Epoch 164/1500
 - 0s - loss: 317.1713 - mae: 13.9362
Epoch 165/1500
 - 0s - loss: 297.1401 - mae: 14.1657
Epoch 166/1500
 - 0s - loss: 258.9710 - mae: 12.5992
Epoch 167/1500
 - 0s - loss: 313.0040 - mae: 14.2678
Epoch 168/1500
 - 0s - loss: 308.4450 - mae: 14.1608
Epoch 169/1500
 - 0s - loss: 327.4249 - mae: 14.4865
Epoch 170/1500
 - 0s - loss: 431.1496 - mae: 16.7164
Epoch 171/1500
 - 0s - loss: 594.2639 - mae: 18.9582
Epoch 172/1500
 - 0s - loss: 393.9583 - mae: 16.1056
Epoch 173/1500
 - 0s - loss: 350.6467 - mae: 15.2250
Epoch 174/1500
 - 0s - loss: 351.4178 - mae: 1

Epoch 314/1500
 - 0s - loss: 54.1353 - mae: 5.4061
Epoch 315/1500
 - 0s - loss: 71.6573 - mae: 6.2761
Epoch 316/1500
 - 0s - loss: 72.4440 - mae: 6.7008
Epoch 317/1500
 - 0s - loss: 49.4524 - mae: 5.2097
Epoch 318/1500
 - 0s - loss: 60.4071 - mae: 5.8519
Epoch 319/1500
 - 0s - loss: 52.7782 - mae: 5.1385
Epoch 320/1500
 - 0s - loss: 46.2339 - mae: 4.7988
Epoch 321/1500
 - 0s - loss: 40.2952 - mae: 4.6250
Epoch 322/1500
 - 0s - loss: 47.1974 - mae: 5.0699
Epoch 323/1500
 - 0s - loss: 49.6839 - mae: 5.2607
Epoch 324/1500
 - 0s - loss: 43.5999 - mae: 4.9740
Epoch 325/1500
 - 0s - loss: 95.3638 - mae: 7.7243
Epoch 326/1500
 - 0s - loss: 75.0228 - mae: 6.7199
Epoch 327/1500
 - 0s - loss: 52.0432 - mae: 5.3316
Epoch 328/1500
 - 0s - loss: 86.9803 - mae: 7.4546
Epoch 329/1500
 - 0s - loss: 77.5802 - mae: 6.3536
Epoch 330/1500
 - 0s - loss: 78.4913 - mae: 6.7703
Epoch 331/1500
 - 0s - loss: 75.4465 - mae: 6.8896
Epoch 332/1500
 - 0s - loss: 73.1939 - mae: 6.7232
Epoch 333/1500
 - 0s - loss: 68

Epoch 474/1500
 - 0s - loss: 110.5869 - mae: 8.4142
Epoch 475/1500
 - 0s - loss: 230.2496 - mae: 13.2460
Epoch 476/1500
 - 0s - loss: 258.2065 - mae: 13.7688
Epoch 477/1500
 - 0s - loss: 248.9413 - mae: 13.6875
Epoch 478/1500
 - 0s - loss: 378.4807 - mae: 17.0946
Epoch 479/1500
 - 0s - loss: 198.7395 - mae: 11.9272
Epoch 480/1500
 - 0s - loss: 147.6154 - mae: 9.8697
Epoch 481/1500
 - 0s - loss: 115.3141 - mae: 8.3468
Epoch 482/1500
 - 0s - loss: 160.3968 - mae: 10.7689
Epoch 483/1500
 - 0s - loss: 148.2794 - mae: 10.0787
Epoch 484/1500
 - 0s - loss: 154.9799 - mae: 10.2607
Epoch 485/1500
 - 0s - loss: 92.8328 - mae: 7.7008
Epoch 486/1500
 - 0s - loss: 159.4552 - mae: 10.6252
Epoch 487/1500
 - 0s - loss: 118.4085 - mae: 8.9306
Epoch 488/1500
 - 0s - loss: 57.3290 - mae: 5.9192
Epoch 489/1500
 - 0s - loss: 99.6482 - mae: 8.1181
Epoch 490/1500
 - 0s - loss: 91.1088 - mae: 7.7406
Epoch 491/1500
 - 0s - loss: 69.7068 - mae: 7.0753
Epoch 492/1500
 - 0s - loss: 47.8337 - mae: 5.6242
Epoch 493

 - 0s - loss: 17.4490 - mae: 3.1572
Epoch 635/1500
 - 0s - loss: 34.3756 - mae: 4.8504
Epoch 636/1500
 - 0s - loss: 41.8983 - mae: 5.4011
Epoch 637/1500
 - 0s - loss: 45.3183 - mae: 5.6729
Epoch 638/1500
 - 0s - loss: 32.4461 - mae: 4.4818
Epoch 639/1500
 - 0s - loss: 22.3767 - mae: 3.6206
Epoch 640/1500
 - 0s - loss: 30.5224 - mae: 4.5027
Epoch 641/1500
 - 0s - loss: 48.0396 - mae: 5.5339
Epoch 642/1500
 - 0s - loss: 49.6311 - mae: 5.8597
Epoch 643/1500
 - 0s - loss: 45.8434 - mae: 5.5854
Epoch 644/1500
 - 0s - loss: 53.3739 - mae: 6.1731
Epoch 645/1500
 - 0s - loss: 96.5999 - mae: 8.7699
Epoch 646/1500
 - 0s - loss: 30.0566 - mae: 4.2312
Epoch 647/1500
 - 0s - loss: 46.3342 - mae: 5.6484
Epoch 648/1500
 - 0s - loss: 36.1387 - mae: 4.8063
Epoch 649/1500
 - 0s - loss: 27.9861 - mae: 4.0607
Epoch 650/1500
 - 0s - loss: 29.0676 - mae: 4.3018
Epoch 651/1500
 - 0s - loss: 16.2052 - mae: 3.1436
Epoch 652/1500
 - 0s - loss: 27.9671 - mae: 4.1418
Epoch 653/1500
 - 0s - loss: 28.7006 - mae: 4.

 - 0s - loss: 17.7345 - mae: 3.2969
Epoch 795/1500
 - 0s - loss: 12.4143 - mae: 2.5719
Epoch 796/1500
 - 0s - loss: 10.1470 - mae: 2.2293
Epoch 797/1500
 - 0s - loss: 12.2906 - mae: 2.6768
Epoch 798/1500
 - 0s - loss: 15.8133 - mae: 3.1401
Epoch 799/1500
 - 0s - loss: 12.4682 - mae: 2.7057
Epoch 800/1500
 - 0s - loss: 15.4563 - mae: 3.1123
Epoch 801/1500
 - 0s - loss: 19.3834 - mae: 3.5037
Epoch 802/1500
 - 0s - loss: 40.8847 - mae: 5.4112
Epoch 803/1500
 - 0s - loss: 18.9199 - mae: 3.3784
Epoch 804/1500
 - 0s - loss: 10.8483 - mae: 2.4123
Epoch 805/1500
 - 0s - loss: 8.6896 - mae: 2.0852
Epoch 806/1500
 - 0s - loss: 8.1692 - mae: 1.9249
Epoch 807/1500
 - 0s - loss: 10.1132 - mae: 2.3048
Epoch 808/1500
 - 0s - loss: 10.3847 - mae: 2.4154
Epoch 809/1500
 - 0s - loss: 7.7195 - mae: 1.9525
Epoch 810/1500
 - 0s - loss: 9.6188 - mae: 2.2476
Epoch 811/1500
 - 0s - loss: 10.2053 - mae: 2.4161
Epoch 812/1500
 - 0s - loss: 19.8257 - mae: 3.5494
Epoch 813/1500
 - 0s - loss: 27.9014 - mae: 4.4909

Epoch 955/1500
 - 0s - loss: 181.5333 - mae: 11.6780
Epoch 956/1500
 - 0s - loss: 148.4591 - mae: 10.3002
Epoch 957/1500
 - 0s - loss: 123.4325 - mae: 9.0403
Epoch 958/1500
 - 0s - loss: 87.0489 - mae: 7.7637
Epoch 959/1500
 - 0s - loss: 67.8560 - mae: 7.2120
Epoch 960/1500
 - 0s - loss: 69.9683 - mae: 6.8903
Epoch 961/1500
 - 0s - loss: 97.7018 - mae: 8.2107
Epoch 962/1500
 - 0s - loss: 72.3632 - mae: 7.1692
Epoch 963/1500
 - 0s - loss: 45.2548 - mae: 5.4221
Epoch 964/1500
 - 0s - loss: 37.9876 - mae: 4.7189
Epoch 965/1500
 - 0s - loss: 48.0289 - mae: 5.7161
Epoch 966/1500
 - 0s - loss: 58.4295 - mae: 6.5429
Epoch 967/1500
 - 0s - loss: 66.3269 - mae: 6.7368
Epoch 968/1500
 - 0s - loss: 114.8011 - mae: 9.2257
Epoch 969/1500
 - 0s - loss: 82.1351 - mae: 7.7856
Epoch 970/1500
 - 0s - loss: 26.9128 - mae: 3.9806
Epoch 971/1500
 - 0s - loss: 21.1658 - mae: 3.4629
Epoch 972/1500
 - 0s - loss: 50.7539 - mae: 5.9495
Epoch 973/1500
 - 0s - loss: 38.2849 - mae: 4.8558
Epoch 974/1500
 - 0s - lo

 - 0s - loss: 32.1907 - mae: 4.3497
Epoch 1114/1500
 - 0s - loss: 37.4594 - mae: 5.1651
Epoch 1115/1500
 - 0s - loss: 64.2113 - mae: 6.9617
Epoch 1116/1500
 - 0s - loss: 49.9692 - mae: 6.0716
Epoch 1117/1500
 - 0s - loss: 28.0817 - mae: 4.0828
Epoch 1118/1500
 - 0s - loss: 19.2850 - mae: 3.3632
Epoch 1119/1500
 - 0s - loss: 22.3249 - mae: 3.8164
Epoch 1120/1500
 - 0s - loss: 16.8281 - mae: 2.9449
Epoch 1121/1500
 - 0s - loss: 18.1907 - mae: 3.3432
Epoch 1122/1500
 - 0s - loss: 18.9923 - mae: 3.5996
Epoch 1123/1500
 - 0s - loss: 14.5243 - mae: 2.9889
Epoch 1124/1500
 - 0s - loss: 20.0966 - mae: 3.6599
Epoch 1125/1500
 - 0s - loss: 16.3709 - mae: 3.2402
Epoch 1126/1500
 - 0s - loss: 10.5965 - mae: 2.4879
Epoch 1127/1500
 - 0s - loss: 11.0614 - mae: 2.5261
Epoch 1128/1500
 - 0s - loss: 15.3016 - mae: 3.1208
Epoch 1129/1500
 - 0s - loss: 18.0741 - mae: 3.4526
Epoch 1130/1500
 - 0s - loss: 20.2966 - mae: 3.7253
Epoch 1131/1500
 - 0s - loss: 13.4964 - mae: 2.7819
Epoch 1132/1500
 - 0s - loss

Epoch 1271/1500
 - 0s - loss: 21.5703 - mae: 3.8387
Epoch 1272/1500
 - 0s - loss: 17.3516 - mae: 3.3459
Epoch 1273/1500
 - 0s - loss: 16.5058 - mae: 3.2131
Epoch 1274/1500
 - 0s - loss: 20.1227 - mae: 3.5921
Epoch 1275/1500
 - 0s - loss: 34.9968 - mae: 4.9141
Epoch 1276/1500
 - 0s - loss: 45.6444 - mae: 5.6009
Epoch 1277/1500
 - 0s - loss: 39.1967 - mae: 5.1582
Epoch 1278/1500
 - 0s - loss: 38.5416 - mae: 5.1678
Epoch 1279/1500
 - 0s - loss: 18.6547 - mae: 3.3951
Epoch 1280/1500
 - 0s - loss: 26.9327 - mae: 4.1492
Epoch 1281/1500
 - 0s - loss: 42.6085 - mae: 5.2512
Epoch 1282/1500
 - 0s - loss: 21.2498 - mae: 3.7130
Epoch 1283/1500
 - 0s - loss: 9.6874 - mae: 2.3047
Epoch 1284/1500
 - 0s - loss: 16.4509 - mae: 3.2360
Epoch 1285/1500
 - 0s - loss: 20.0542 - mae: 3.6521
Epoch 1286/1500
 - 0s - loss: 17.5154 - mae: 3.4391
Epoch 1287/1500
 - 0s - loss: 13.9042 - mae: 2.9726
Epoch 1288/1500
 - 0s - loss: 25.4371 - mae: 4.1057
Epoch 1289/1500
 - 0s - loss: 15.8818 - mae: 3.2005
Epoch 1290/15

Epoch 1429/1500
 - 0s - loss: 13.9650 - mae: 2.8941
Epoch 1430/1500
 - 0s - loss: 12.4687 - mae: 2.7856
Epoch 1431/1500
 - 0s - loss: 9.8375 - mae: 2.3942
Epoch 1432/1500
 - 0s - loss: 7.8175 - mae: 2.0831
Epoch 1433/1500
 - 0s - loss: 8.1855 - mae: 2.1110
Epoch 1434/1500
 - 0s - loss: 10.2369 - mae: 2.4671
Epoch 1435/1500
 - 0s - loss: 9.9685 - mae: 2.5008
Epoch 1436/1500
 - 0s - loss: 6.2582 - mae: 1.7987
Epoch 1437/1500
 - 0s - loss: 10.1430 - mae: 2.4774
Epoch 1438/1500
 - 0s - loss: 6.4815 - mae: 1.8440
Epoch 1439/1500
 - 0s - loss: 10.0417 - mae: 2.4445
Epoch 1440/1500
 - 0s - loss: 8.2807 - mae: 2.1263
Epoch 1441/1500
 - 0s - loss: 9.3157 - mae: 2.3266
Epoch 1442/1500
 - 0s - loss: 8.3159 - mae: 2.2136
Epoch 1443/1500
 - 0s - loss: 10.9796 - mae: 2.6948
Epoch 1444/1500
 - 0s - loss: 19.2303 - mae: 3.5877
Epoch 1445/1500
 - 0s - loss: 13.9865 - mae: 2.9508
Epoch 1446/1500
 - 0s - loss: 9.0580 - mae: 2.3250
Epoch 1447/1500
 - 0s - loss: 9.6798 - mae: 2.4816
Epoch 1448/1500
 - 0s -

## Оценка точности нейронной сети

In [70]:
mse, mae = model.evaluate(X_test, y_test, verbose=0)

In [71]:
pred = model.predict(X_test)

In [72]:
print("Средняя абсолютная ошибка:", mae)

Средняя абсолютная ошибка: 13.794415473937988


## Проверка на тестовых данных

In [80]:
print("Предсказанное значение:", pred[1][0], ", правильное значение:", y_test[1])
print("Предсказанное значение:", pred[2][0], ", правильное значение:", y_test[2])
print("Предсказанное значение:", pred[3][0], ", правильное значение:", y_test[3])
print("Предсказанное значение:", pred[4][0], ", правильное значение:", y_test[4])
print("Предсказанное значение:", pred[10][0], ", правильное значение:", y_test[10])
print("Предсказанное значение:", pred[12][0], ", правильное значение:", y_test[12])
print("Предсказанное значение:", pred[14][0], ", правильное значение:", y_test[14])
print("Предсказанное значение:", pred[16][0], ", правильное значение:", y_test[16])
print("Предсказанное значение:", pred[18][0], ", правильное значение:", y_test[18])
print("Предсказанное значение:", pred[30][0], ", правильное значение:", y_test[30])
print("Предсказанное значение:", pred[35][0], ", правильное значение:", y_test[35])
print("Предсказанное значение:", pred[40][0], ", правильное значение:", y_test[40])
print("Предсказанное значение:", pred[45][0], ", правильное значение:", y_test[45])
print("Предсказанное значение:", pred[50][0], ", правильное значение:", y_test[50])
print("Предсказанное значение:", pred[60][0], ", правильное значение:", y_test[60])

Предсказанное значение: 444.43744 , правильное значение: 436.0
Предсказанное значение: 405.27295 , правильное значение: 404.0
Предсказанное значение: 431.02792 , правильное значение: 431.0
Предсказанное значение: 492.26572 , правильное значение: 489.0
Предсказанное значение: 453.63623 , правильное значение: 455.0
Предсказанное значение: 72.30005 , правильное значение: 71.0
Предсказанное значение: 152.31105 , правильное значение: 151.0
Предсказанное значение: 242.86726 , правильное значение: 241.0
Предсказанное значение: 484.5351 , правильное значение: 480.0
Предсказанное значение: 418.8023 , правильное значение: 416.0
Предсказанное значение: 380.41943 , правильное значение: 370.0
Предсказанное значение: 434.111 , правильное значение: 436.0
Предсказанное значение: 297.26907 , правильное значение: 251.0
Предсказанное значение: 476.46594 , правильное значение: 465.0
Предсказанное значение: 162.23152 , правильное значение: 157.0
